In [28]:
import pandas as pd
import requests
import json

def statement(ticker,statement,period):
    if statement =='IS':
        temp='income-statement'
    elif statement =='BS':
        temp='balance-sheet-statement'
    else:
        temp='cash-flow-statement'
    if period=='Q':
        temp2 = 'quarter'
    else:
        temp2 = 'annual'
    bs = requests.get(f'https://financialmodelingprep.com/api/v3/{temp}/{ticker}?period={temp2}&limit=400&apikey=28e38c7ed1ebc463e4f6027b2bff225a')
    bs = bs.json()
    
    bs = pd.DataFrame.from_dict(bs)
    bs = bs.T
    bs = bs.drop(['symbol','finalLink','link','reportedCurrency','fillingDate','acceptedDate','period'])
    bs.columns = bs.iloc[0]

    bs = bs.iloc[1:,:4]
    
    return bs

In [29]:
def net_income(ticker):
    df=pd.DataFrame(statement(ticker,'IS','A'))
    return float(df.loc['netIncome'][0])

def roa(ticker):
    df=pd.DataFrame(statement(ticker,'BS','A'))
    p,q=float(df.loc['totalAssets'][0]),float(df.loc['totalAssets'][1])
    av_assets=(p+q)/2
    return net_income(ticker)/av_assets

def ocf(ticker):
    df=pd.DataFrame(statement(ticker,'CF','A'))
    return float(df.loc['operatingCashFlow'][0])

def ltdebt(ticker):
    df=pd.DataFrame(statement(ticker,'BS','A'))
    p,q=float(df.loc['longTermDebt'][0]),float(df.loc['longTermDebt'][1])
    return q-p

def current_ratio(ticker):
    df=pd.DataFrame(statement(ticker,'BS','A'))
    p=float(df.loc['totalCurrentAssets'][0])
    q=float(df.loc['totalCurrentAssets'][1])
    r=float(df.loc['totalCurrentLiabilities'][0])
    s=float(df.loc['totalCurrentLiabilities'][1])
    current_ratio1=p/r
    current_ratio2=q/s
    return current_ratio1-current_ratio2
   
def new_shares(ticker):
    df=pd.DataFrame(statement(ticker,'BS','A'))
    p,q=float(df.loc['retainedEarnings'][0]),float(df.loc['retainedEarnings'][1])
    r,s=float(df.loc['totalStockholdersEquity'][0]),float(df.loc['totalStockholdersEquity'][1])
    return (s-q)-(r-p)

def gross_margin(ticker):
    df=pd.DataFrame(statement(ticker,'IS','A'))
    p,q=float(df.loc['grossProfit'][0]),float(df.loc['grossProfit'][1])
    return p-q

def Asset_Turnover_Ratio(ticker):
    df1=pd.DataFrame(statement(ticker,'IS','A'))
    df2=pd.DataFrame(statement(ticker,'BS','A'))
    p,q,s=float(df2.loc['totalAssets'][0]),float(df2.loc['totalAssets'][1]),float(df2.loc['totalAssets'][2])
    av_assets1=(p+q)/2
    av_assets2=(q+s)/2
    atr1=float(df1.loc['revenue'][0])/av_assets1
    atr2=float(df1.loc['revenue'][1])/av_assets2
    return atr1-atr2

def Piotroski_Score(ticker):
    Pscore=0
    if net_income(ticker)>0:
        Pscore+=1
    if roa(ticker)>0:
        Pscore+=1
    if ocf(ticker)>0:
        Pscore+=1
    if ocf(ticker)>net_income(ticker):
        Pscore+=1
    if ltdebt(ticker)>0:
        Pscore+=1
    if current_ratio(ticker)>0:
        Pscore+=1
    if new_shares(ticker)>0:
        Pscore+=1
    if gross_margin(ticker)>0:
        Pscore+=1
    if Asset_Turnover_Ratio(ticker)>0:
        Pscore+=1
    return Pscore

In [30]:
tickers=['AAPL','GOOG','TSLA','F','MSFT']
data2={}
for ticker in tickers:
    data2[ticker]=Piotroski_Score(ticker)
    data=pd.DataFrame(data2,index=[0])    
data

,AAPL,GOOG,TSLA,F,MSFT
0,6,5,7,4,7
